In [1]:
#import some libraries
import nltk
from pprint import pprint
import csv
from typing import List
import re
from collections import Counter
import numpy as np
import pandas as pd
from IPython.display import clear_output
import pickle
from sklearn.externals import joblib

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble, cluster
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from nltk.tokenize import word_tokenize,sent_tokenize

from nltk.corpus import stopwords # library 
nltk.download('stopwords')
all_stopwords = set(stopwords.words('english')) # set the language 
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nagem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# #remove stop-words
# from nltk.corpus import stopwords # library 
# nltk.download('stopwords')
# all_stopwords = set(stopwords.words('english')) # set the language 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nagem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#load datset
df = pd.read_csv('C:/Users/nagem/OneDrive/Documents/MSBA 2019/570/clash_of_clans_570/clash-of-clans.csv')
reviews_data = pd.DataFrame({'review': df.Content, 'sentiment': df.Rating})
print(reviews_data.head(2))

# reading review data with panda frames 
reviews_data.describe()


review  sentiment
0               💜💜💜          5
1  Pls make it th13          3


,sentiment
count,50001.000000
mean,4.612628
std,1.026489
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


The dataset was loaded and 2 columns were kept, the one with reviews and the sentiment score which was on a scale of 1 to 5.

In [16]:
# encode non-anscii to anscii and then decode string
reviews_data['review'] = reviews_data['review'].apply(lambda x: str(x).encode('ascii','ignore').decode())

The reviews had some non-anscii characters which had to be encoded and decoded.

In [18]:
# The sentiments are either 'positive' or 'negative' and are evenly distributed. Lets preprocess the text using the simple tokenizer we built in last class. We call it preprocess_text now.
def preprocess_text(text):
    pattern1 = re.compile("<br /><br />|\.")
    lines = re.split(pattern1, text)
    tokens = []
    for line in lines:
        tokens += line.split(" ")

    # lowercase and remove any non-alphanumeric characters from tokens for normalize
    normalized_tokens = [token for token in tokens if re.search(r"^[A-Za-z]\w*$", token) is not None]
    return  " ".join([
            token
            for token in normalized_tokens
            if token and token not in all_stopwords and len(tokens) > 1
        ])
    

  
custom_review = "I hated the film. It was a disaster. Poor direction, bad acting."
custom_review_tokens = preprocess_text(custom_review)
print(custom_review_tokens)

I hated film It disaster Poor bad acting


The reviews were cleaned and split into words. Normalization removed punctuation and stopwords. 

In [19]:
#apply preprocessing to review data
reviews_data['review'] = reviews_data['review'].apply(preprocess_text)

In [21]:
lower_count_thr = 1000 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens

vectorizer=TfidfVectorizer(min_df=lower_count_thr,max_df=upper_count_thr,binary=False,ngram_range=(1,1))

Initialy the TfidVectorizer was tried with a lower threshold of 1000 and an upper count of 5000. The ngram_range was set to (1,1).

In [22]:
tfidf_vectors = vectorizer.fit_transform(reviews_data['review'] )

In [23]:
feature_names = vectorizer.get_feature_names()
dense = tfidf_vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [24]:
print(len(df))
print(df[10:30])

50001
    awesome  best       coc  ever  great   it     like  love  much  play  \
10      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
11      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
12      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
13      0.0   0.0  0.000000   0.0    0.0  0.0  0.64249   0.0   0.0   0.0   
14      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
15      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
16      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
17      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
18      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
19      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
20      0.0   0.0  0.000000   0.0    1.0  0.0  0.00000   0.0   0.0   0.0   
21      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   1.0   0.0   0.0   
22    

In [31]:
num_clust = 3
km = KMeans(n_clusters = num_clust, max_iter=100, n_init=1)

In [32]:
km.fit(tfidf_vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [33]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clust):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 it
 best
 play
 time
 like
 love
 world
 awesome
 great
 ever
Cluster 1:
 best
 like
 this
 very
 play
 time
 ever
 super
 great
 awesome
Cluster 2:
 love
 much
 coc
 this
 play
 best
 supercell
 like
 awesome
 great


The resulting clusters were very positive and somewhat similar sounding. 

In [83]:
#Count vectorizer
def run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter=100, n_init=1): 
    vectorizer=TfidfVectorizer(min_df=lower_count_thr,max_df=upper_count_thr,binary=False,ngram_range=(n_gram_min,n_gram_max))

    tfidf_vectors = vectorizer.fit_transform(reviews)

    feature_names = vectorizer.get_feature_names()
    dense = tfidf_vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    print('Length of denselist:\n', len(df))
    print('#10-30 from denselist:\n',df[10:30])

    km = KMeans(n_clusters = num_clust, max_iter=max_iter, n_init=n_init)

    km.fit(tfidf_vectors)

    print("Top terms per cluster:")
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num_clust):
        print("\nCluster %d:" % i),
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind]),
        print

A function was created to run multiple Kmeans with one cell that had adjustable parameters. Different variations were tried to see which would result in useful clusters. 

In [37]:
lower_count_thr = 500 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Length of denselist:
 50001
#10-30 from denselist:
     addictive  also  amazing  app  awesome  bad  best  better  clan  clans  \
10    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
11    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
12    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
13    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
14    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
15    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
16    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
17    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
18    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
19    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
20    0.00000   0.0      0.0  0.0      0.0  0.0   0.0     0.0   0.0    0.0   
21    0.0000

Since and overwhelming percentage of reviews were 5 star and had similar words, the lower and upper threshold were lowered to captured more unique words. 

In [39]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 1000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Length of denselist:
 50001
#10-30 from denselist:
     able  account       add  addicted  addicting  addictive  all  also  \
10   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
11   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
12   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
13   0.0      0.0  0.505035       0.0        0.0   0.000000  0.0   0.0   
14   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
15   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
16   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
17   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
18   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
19   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
20   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
21   0.0      0.0  0.000000       0.0        0.0   0.000000 

In [40]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Length of denselist:
 50001
#10-30 from denselist:
     able  account       add  addicted  addicting  addictive  all  also  \
10   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
11   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
12   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
13   0.0      0.0  0.444277       0.0        0.0   0.000000  0.0   0.0   
14   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
15   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
16   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
17   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
18   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
19   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
20   0.0      0.0  0.000000       0.0        0.0   0.000000  0.0   0.0   
21   0.0      0.0  0.000000       0.0        0.0   0.000000 

Experiments with different number of ngrams were tested to see if more information could be interpreted from the clusters.

In [41]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 2
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Length of denselist:
 50001
#10-30 from denselist:
     able  account       add  addicted  addicting  addictive  addictive game  \
10   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
11   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
12   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
13   0.0      0.0  0.444277       0.0        0.0   0.000000             0.0   
14   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
15   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
16   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
17   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
18   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
19   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
20   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   


In [43]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 2
reviews = reviews_data['review']
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

Length of denselist:
 50001
#10-30 from denselist:
     able  account       add  addicted  addicting  addictive  addictive game  \
10   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
11   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
12   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
13   0.0      0.0  0.444277       0.0        0.0   0.000000             0.0   
14   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
15   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
16   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
17   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
18   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
19   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   
20   0.0      0.0  0.000000       0.0        0.0   0.000000             0.0   


In [49]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 800 # frequent/common tokens
num_clust = 5
n_gram_min = 3
n_gram_max = 3
reviews = reviews_data['review']
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

Length of denselist:
 50001
#10-30 from denselist:
     best game world  best strategy game  game best game  game ever played  \
10              0.0                 0.0             0.0               0.0   
11              0.0                 0.0             0.0               0.0   
12              0.0                 0.0             0.0               0.0   
13              0.0                 0.0             0.0               0.0   
14              0.0                 0.0             0.0               0.0   
15              0.0                 0.0             0.0               0.0   
16              0.0                 0.0             0.0               0.0   
17              0.0                 0.0             0.0               0.0   
18              0.0                 0.0             0.0               0.0   
19              0.0                 0.0             0.0               0.0   
20              0.0                 0.0             0.0               0.0   
21              0.0     

In [57]:
rating1 = reviews_data[reviews_data['sentiment']==1]['review']
print('Length rating1: ', len(rating1))

rating2 = reviews_data[reviews_data['sentiment']==2]['review']
print('Length rating2: ', len(rating2))

rating3 = reviews_data[reviews_data['sentiment']==3]['review']
print('Length rating3: ', len(rating3))

rating4 = reviews_data[reviews_data['sentiment']==4]['review']
print('Length rating4: ', len(rating4))

rating5 = reviews_data[reviews_data['sentiment']==5]['review']
print('Length rating5: ', len(rating5))

Length rating1:  2877
Length rating2:  583
Length rating3:  1398
Length rating4:  3316
Length rating5:  41827


The reviews were then separated into groups based on rating and clusters made of each group.

## 1 star rating

In [69]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.9) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 2
n_gram_max = 4
reviews = rating1
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

3 2589
Length of denselist:
 2877
#10-30 from denselist:
     able login  able open  able open account  able play  account back  \
10         0.0        0.0                0.0        0.0           0.0   
11         0.0        0.0                0.0        0.0           0.0   
12         0.0        0.0                0.0        0.0           0.0   
13         0.0        0.0                0.0        0.0           0.0   
14         0.0        0.0                0.0        0.0           0.0   
15         0.0        0.0                0.0        0.0           0.0   
16         0.0        0.0                0.0        0.0           0.0   
17         0.0        0.0                0.0        0.0           0.0   
18         0.0        0.0                0.0        0.0           0.0   
19         0.0        0.0                0.0        0.0           0.0   
20         0.0        0.0                0.0        0.0           0.0   
21         0.0        0.0                0.0        0.0           

The one star rating had a large amount of negative words and some positive. Of the three clusters above, two were related to supercell sugesting this may be a factor in disatisfied users. 

In [73]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating1
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 2302
Length of denselist:
 2877
#10-30 from denselist:
     a1 android pie  aa gaya bori  aa gaya bori bistar  aa jate wah  \
10             0.0           0.0                  0.0          0.0   
11             0.0           0.0                  0.0          0.0   
12             0.0           0.0                  0.0          0.0   
13             0.0           0.0                  0.0          0.0   
14             0.0           0.0                  0.0          0.0   
15             0.0           0.0                  0.0          0.0   
16             0.0           0.0                  0.0          0.0   
17             0.0           0.0                  0.0          0.0   
18             0.0           0.0                  0.0          0.0   
19             0.0           0.0                  0.0          0.0   
20             0.0           0.0                  0.0          0.0   
21             0.0           0.0                  0.0          0.0   
22             0.0           0.0

We hypothesized that the majority of reviews in any rating category would be farely generic so further lowered the threshold  in hopes of capturing more unique and descriptive words. When the cluster focused more on rare words and use ngram of (3,4), we began to see some interesting things come out. In the clusters above, the cluster 0 seems to have pretty generic negative reviews. For cluster 1, one of the commonly repeated thems is devs, suggesting the players blame the developers for the negative state of the game. Cluster 2 references the tropps and battles with the emphasis on how unpredictable of inconsistent they were. 

In [85]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.45) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 5
n_gram_min = 3
n_gram_max = 4
reviews = rating1
max_iter = 50
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

3 1295
Length of denselist:
 2877
#10-30 from denselist:
     able open account  account try recover  back google play  bad game bad  \
10                0.0                  0.0               0.0           0.0   
11                0.0                  0.0               0.0           0.0   
12                0.0                  0.0               0.0           0.0   
13                0.0                  0.0               0.0           0.0   
14                0.0                  0.0               0.0           0.0   
15                0.0                  0.0               0.0           0.0   
16                0.0                  0.0               0.0           0.0   
17                0.0                  0.0               0.0           0.0   
18                0.0                  0.0               0.0           0.0   
19                0.0                  0.0               0.0           0.0   
20                0.0                  0.0               0.0           0.0   
21    

A somewhat frequent occurance in the last few clusters was mention of google or google play. There maybe some aspect of google play that is causing users to have a bad experience. Another common occurance references "id much upgraded" or "id much upgraded face".

## 2 star rating

In [87]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.5) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating2
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 292
Length of denselist:
 583
#10-30 from denselist:
     aand plz add  aand plz add wall  able finish fed  able finish fed point  \
10           0.0                0.0              0.0                    0.0   
11           0.0                0.0              0.0                    0.0   
12           0.0                0.0              0.0                    0.0   
13           0.0                0.0              0.0                    0.0   
14           0.0                0.0              0.0                    0.0   
15           0.0                0.0              0.0                    0.0   
16           0.0                0.0              0.0                    0.0   
17           0.0                0.0              0.0                    0.0   
18           0.0                0.0              0.0                    0.0   
19           0.0                0.0              0.0                    0.0   
20           0.0                0.0              0.0                    0.0

In [88]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating2
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 466
Length of denselist:
 583
#10-30 from denselist:
     aand plz add  aand plz add wall  able finish fed  able finish fed point  \
10           0.0                0.0              0.0                    0.0   
11           0.0                0.0              0.0                    0.0   
12           0.0                0.0              0.0                    0.0   
13           0.0                0.0              0.0                    0.0   
14           0.0                0.0              0.0                    0.0   
15           0.0                0.0              0.0                    0.0   
16           0.0                0.0              0.0                    0.0   
17           0.0                0.0              0.0                    0.0   
18           0.0                0.0              0.0                    0.0   
19           0.0                0.0              0.0                    0.0   
20           0.0                0.0              0.0                    0.0

Looking at the two star reviews, we see a some different topics occuring. In the clusters above, cluster 0 references money which may be related to the game requiring players to spend money in order to get better weapons. They are able to achieve it but it takes a very long time. cluster 2 references how long it takes to achieve nakakawala, again related to the time required to get certain things without spending money. 

## 3 star rating

In [89]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 466
Length of denselist:
 1398
#10-30 from denselist:
     aa lloonngg mmiissmmaattcchh  aa lloonngg mmiissmmaattcchh like  \
10                           0.0                                0.0   
11                           0.0                                0.0   
12                           0.0                                0.0   
13                           0.0                                0.0   
14                           0.0                                0.0   
15                           0.0                                0.0   
16                           0.0                                0.0   
17                           0.0                                0.0   
18                           0.0                                0.0   
19                           0.0                                0.0   
20                           0.0                                0.0   
21                           0.0                                0.0   
22                  

In [ ]:
At 3 stars we start to see more positive sentiment again. In the above clusters, cluster 0 has pretty positive sentiment. Cluster 1 seems to talk about difficulties getting enough loot. Cluster 2 appears to be related to users that felt the main had become stale. 

In [92]:
lower_count_thr = round(len(reviews)*.0001) # rare words/tokens
upper_count_thr = round(len(reviews)*.6) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 2
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

0 839
Length of denselist:
 1398
#10-30 from denselist:
     aa lloonngg mmiissmmaattcchh  aa lloonngg mmiissmmaattcchh like  \
10                           0.0                                0.0   
11                           0.0                                0.0   
12                           0.0                                0.0   
13                           0.0                                0.0   
14                           0.0                                0.0   
15                           0.0                                0.0   
16                           0.0                                0.0   
17                           0.0                                0.0   
18                           0.0                                0.0   
19                           0.0                                0.0   
20                           0.0                                0.0   
21                           0.0                                0.0   
22                  

Another clustering of 3 start reviews had a class related to progress restart of deleted.

In [95]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 1118
Length of denselist:
 1398
#10-30 from denselist:
     aa lloonngg mmiissmmaattcchh  aa lloonngg mmiissmmaattcchh like  \
10                           0.0                                0.0   
11                           0.0                                0.0   
12                           0.0                                0.0   
13                           0.0                                0.0   
14                           0.0                                0.0   
15                           0.0                                0.0   
16                           0.0                                0.0   
17                           0.0                                0.0   
18                           0.0                                0.0   
19                           0.0                                0.0   
20                           0.0                                0.0   
21                           0.0                                0.0   
22                 

In cluster 3 there are references to when connection was very bad during Christmas. 

In [101]:
lower_count_thr = round(len(reviews)*.0015) # rare words/tokens
upper_count_thr = round(len(reviews)*.65) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

2 909
Length of denselist:
 1398
#10-30 from denselist:
     account connected id  account connected id gmail  account please give  \
10                   0.0                         0.0                  0.0   
11                   0.0                         0.0                  0.0   
12                   0.0                         0.0                  0.0   
13                   0.0                         0.0                  0.0   
14                   0.0                         0.0                  0.0   
15                   0.0                         0.0                  0.0   
16                   0.0                         0.0                  0.0   
17                   0.0                         0.0                  0.0   
18                   0.0                         0.0                  0.0   
19                   0.0                         0.0                  0.0   
20                   0.0                         0.0                  0.0   
21                 

In the clusters above, we again see gmail account mentioned. There may be issues that users are experiencing related to google or gmail accounts. 

## 4 star rating

In [108]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.4) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating4
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

2 1326
Length of denselist:
 3316
#10-30 from denselist:
     able open game  account please help  add christmas theme  \
10             0.0                  0.0                  0.0   
11             0.0                  0.0                  0.0   
12             0.0                  0.0                  0.0   
13             0.0                  0.0                  0.0   
14             0.0                  0.0                  0.0   
15             0.0                  0.0                  0.0   
16             0.0                  0.0                  0.0   
17             0.0                  0.0                  0.0   
18             0.0                  0.0                  0.0   
19             0.0                  0.0                  0.0   
20             0.0                  0.0                  0.0   
21             0.0                  0.0                  0.0   
22             0.0                  0.0                  0.0   
23             0.0                  0.0       

With the 4 star ratings, we seem to have more positive sentiment than the previous ratings. Two new topics also appear. One refers to children and the other to dark or gold elixir.

In [ ]:
## 5 star rating

In [110]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.3) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating5
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

2 995
Length of denselist:
 41827
#10-30 from denselist:
     a1 game world  able join clan  able play friends  able to understand  \
10            0.0             0.0                0.0                 0.0   
11            0.0             0.0                0.0                 0.0   
12            0.0             0.0                0.0                 0.0   
13            0.0             0.0                0.0                 0.0   
14            0.0             0.0                0.0                 0.0   
15            0.0             0.0                0.0                 0.0   
16            0.0             0.0                0.0                 0.0   
17            0.0             0.0                0.0                 0.0   
18            0.0             0.0                0.0                 0.0   
19            0.0             0.0                0.0                 0.0   
20            0.0             0.0                0.0                 0.0   
21            0.0             

The 5 star ratings were very positive in tone and did not provide much information on details about things that people liked or things that could be improved. 

The purpose of this research was in part to determine features of the game that players either liked or did not like so the developers could make improvements in those areas. 

* From one star ratings we see that players are likely unhappy about unpredictable troops and battle. 
* Two star reviews feel negatively about how long it takes to get further in the game without spending money. 
* For three star reviews the clusters suggest that players are having issue with their gmail or google play accounts. 
* Four star reviews reference dark or gold elixir it is seen with references to better and children. 
* The five star reviews did not seem to reference specific as the other ratings did. However, these were the majority of the reviews which suggests the game is well liked and negative things mentioned in lower star reviews do not greatly impact the majority of players badly. 